In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
## AWS
import boto3
import awswrangler as wr

# load secret keys
db_host = os.environ.get('DB_HOST')
db_name = os.environ.get('DB_NAME')
db_user = os.environ.get('DB_USER')
db_pass = os.environ.get('DB_PASS')
db_port = os.environ.get('DB_PORT')

In [39]:
conn = psycopg2.connect(database=db_name,
user=db_user,
password=db_pass,
host=db_host,
port=db_port)
conn.set_session(autocommit=True)
cur = conn.cursor()

In [40]:
# load data
# helper function
def bucket_raw_path(bucket_name,path_dir):
    '''get raw path of bucket'''
    raw_path = f's3://{bucket_name}/{path_dir}'
    return raw_path


def load_data(raw_s3):
    # depends what file you are uploading
    file = wr.s3.list_objects(raw_s3)[1]  # depends what file you are uploading
    return wr.s3.read_csv(file)

def create_table_from_df(dataframe,column_name,new_col_name,new_pk_name):
    """
    """
    col_data_list = dataframe[column_name].unique().tolist()
    total_rows = len(col_data_list)
    df_table = pd.DataFrame(col_data_list,columns=[new_col_name])
    df_table = df_table.reset_index()
    df_table.rename(columns={'index':new_pk_name}, inplace=True)
    df_table[new_pk_name] = df_table[new_pk_name] +1
    return df_table




def drop_add_pk(data,data_table,lo,ro):
    data = data.merge(data_table, left_on=lo, right_on=ro,suffixes=(True,True))
    data.drop([lo,ro], axis=1,inplace=True)
    return data

# LOAD DATA

In [91]:
raw = bucket_raw_path('dend-data',f'capstone/load-data/')
wr.s3.list_objects(raw)

['s3://dend-data/capstone/load-data/crime-weather-09-18.csv',
 's3://dend-data/capstone/load-data/crime-weather-sample-100-09-18.csv',
 's3://dend-data/capstone/load-data/crime-weather-sample-1000-09-18.csv']

In [92]:
wr.s3.list_objects(raw)[1]

's3://dend-data/capstone/load-data/crime-weather-sample-100-09-18.csv'

In [94]:
raw

's3://dend-data/capstone/load-data/'

In [93]:
%%time
data = load_data(raw)

CPU times: user 111 ms, sys: 32.9 ms, total: 144 ms
Wall time: 1.11 s


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date_time            100 non-null    object 
 1   offenses             100 non-null    int64  
 2   offense_type         100 non-null    object 
 3   block_range          100 non-null    object 
 4   street_name          100 non-null    object 
 5   type                 100 non-null    object 
 6   suffix               100 non-null    object 
 7   beat                 100 non-null    object 
 8   premise              100 non-null    object 
 9   date                 100 non-null    object 
 10  hour                 100 non-null    int64  
 11  year                 100 non-null    int64  
 12  premise_description  100 non-null    object 
 13  temp                 100 non-null    float64
 14  feels_like           100 non-null    float64
 15  temp_min             100 non-null    floa

In [45]:
data.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,...,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description
0,2018-04-13 19:00:00,1,Theft,5100-5199,buffalo speedway,-,-,1A50,Restaurant or Cafeteria Parking Lot,04/13/2018,...,84.650,80.654,83.102,59,7.7,0.56,0.0,75,Rain,light rain
1,2009-11-10 10:00:00,1,Burglary,5400-5499,lymbar,dr,-,15E10,20R,11/10/09 00:00:00,...,63.500,62.654,64.418,92,4.1,0.00,0.0,1,Clear,sky is clear
2,2016-02-27 17:00:00,1,Burglary,6300-6399,bellfort,st,W,17E40,Apartment,2016-02-27 00:00:00,...,64.184,64.094,66.902,31,2.1,0.00,0.0,1,Clear,sky is clear
3,2013-10-26 17:00:00,1,Theft,7900-7999,wallisville,rd,-,9C20,250,2013-10-26 00:00:00,...,76.406,73.616,78.818,62,5.7,0.00,0.0,1,Clear,sky is clear
4,2018-03-10 00:00:00,1,Robbery,8900-8999,braesmont,dr,-,15E10,Apartment Parking Lot,03/10/2018,...,68.036,68.000,70.016,65,4.1,0.00,0.0,90,Clouds,overcast clouds


# remove unused columns

In [46]:
data.columns.tolist()

['date_time',
 'offenses',
 'offense_type',
 'block_range',
 'street_name',
 'type',
 'suffix',
 'beat',
 'premise',
 'date',
 'hour',
 'year',
 'premise_description',
 'temp',
 'feels_like',
 'temp_min',
 'temp_max',
 'humidity_per',
 'wind_speed',
 'rain_vol_1h_mm',
 'snow_vol_1h_mm',
 'clouds_all_per',
 'weather_main',
 'weather_description']

In [47]:
data = data[['date_time',
 'offenses',
 'offense_type',
 'block_range',
 'street_name',
 'beat',
 'premise_description',
 'temp',
 'feels_like',
 'humidity_per',
 'rain_vol_1h_mm',
 'snow_vol_1h_mm']]

# CREATE TABLES FROM DF

## CREATE offense_table

In [48]:
offense_table = create_table_from_df(data,'offense_type','offense_name','offense_pk')
offense_table.head()

,offense_pk,offense_name
0,1,Theft
1,2,Burglary
2,3,Robbery
3,4,Aggravated Assault
4,5,Auto Theft


In [49]:
data.sample()

,date_time,offenses,offense_type,block_range,street_name,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm
27,2013-07-04 17:00:00,1,Auto Theft,9500-9599,post oak,15E10,strip business center parking lot,88.862,90.662,45,0.0,0.0


In [50]:
data = drop_add_pk(data,offense_table,'offense_type','offense_name')

## CREATE police_beat_table

In [51]:
police_beat_table = create_table_from_df(data,'beat','beat_name','beat_pk')
police_beat_table.head()

,beat_pk,beat_name
0,1,1A50
1,2,9C20
2,3,18F30
3,4,15E10
4,5,4F10


## remove columns & add pk col

In [52]:
data = drop_add_pk(data,police_beat_table,'beat','beat_name')
data.sample()

,date_time,offenses,block_range,street_name,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk
29,2011-04-15 00:00:00,1,10500-10599,valley forge,apartment,77.594,78.35,70,0.0,0.0,1,11


## CREATE premise_description_table

In [53]:
premise_table = create_table_from_df(data,'premise_description','premise_name','premise_pk')
premise_table.head()

,premise_pk,premise_name
0,1,restaurant or cafeteria parking lot
1,2,other unknown or not listed
2,3,bar night club parking lot
3,4,unk
4,5,other unknown


## remove col & add pk col

In [54]:
data = drop_add_pk(data,premise_table,'premise_description','premise_name')
data.sample()

,date_time,offenses,block_range,street_name,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk
82,2013-07-31 22:00:00,1,2300-2399,louisiana,99.446,104.432,35,0.0,0.0,1,30,26


In [55]:
data.head()

,date_time,offenses,block_range,street_name,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk
0,2018-04-13 19:00:00,1,5100-5199,buffalo speedway,82.292,84.650,59,0.56,0.0,1,1,1
1,2015-08-21 22:00:00,1,2400-2499,richmond,90.914,97.358,52,0.00,0.0,1,32,1
2,2016-08-11 17:00:00,1,3800-3899,richmond,93.434,106.034,58,0.10,0.0,1,1,2
3,2013-09-13 17:00:00,1,3400-3499,alabama,91.544,94.712,44,0.00,0.0,1,1,3
4,2014-07-02 07:00:00,1,3600-3699,alabama,79.124,79.124,85,0.00,0.0,2,1,4


# CREATE address_table

In [56]:
# clenup data
data.block_range.replace('UNK','10-100',inplace=True)
data.block_range.replace('1.1103e+006-1.1104e+006','10-100',inplace=True)

def block_range_split(df):
    '''split blockrange col values
    then give median value as a string'''
    first = df.block_range.str.split(pat='-',expand=True)[0].astype('int')
    second = df.block_range.str.split(pat='-',expand=True)[1].astype('int')
    med = np.ceil((second + first)/2).astype('int')
    med = med.astype('str')
    street = df.street_name
    res = med + " " + street +" Houston, TX"
    #res = f"{med}{street} Houston, TX"
    return res


data['og_address'] = block_range_split(data)
# drop street_name & block range
data.drop(['block_range','street_name'], axis=1,inplace=True)

In [57]:
data.head()

,date_time,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,og_address
0,2018-04-13 19:00:00,1,82.292,84.650,59,0.56,0.0,1,1,1,"5150 buffalo speedway Houston, TX"
1,2015-08-21 22:00:00,1,90.914,97.358,52,0.00,0.0,1,32,1,"2450 richmond Houston, TX"
2,2016-08-11 17:00:00,1,93.434,106.034,58,0.10,0.0,1,1,2,"3850 richmond Houston, TX"
3,2013-09-13 17:00:00,1,91.544,94.712,44,0.00,0.0,1,1,3,"3450 alabama Houston, TX"
4,2014-07-02 07:00:00,1,79.124,79.124,85,0.00,0.0,2,1,4,"3650 alabama Houston, TX"


In [58]:
address_table = create_table_from_df(data,'og_address','full_address','address_pk')
address_table.head()

,address_pk,full_address
0,1,"5150 buffalo speedway Houston, TX"
1,2,"2450 richmond Houston, TX"
2,3,"3850 richmond Houston, TX"
3,4,"3450 alabama Houston, TX"
4,5,"3650 alabama Houston, TX"


In [59]:
## 

In [60]:
data = drop_add_pk(data,address_table,'og_address','full_address')

data.sample()

,date_time,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,address_pk
14,2011-07-17 23:00:00,1,98.204,100.922,33,0.0,0.0,5,53,6,15


# CREATE datetime_table

In [61]:
dt_table = create_table_from_df(data,'date_time','date_time','date_time_pk')

# inster time data records
t = pd.to_datetime(dt_table['date_time'])

time_data = (dt_table.date_time_pk,dt_table.date_time,t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year,t.dt.weekday)
column_labels = ('date_time_pk','date_time', 'hour', 'day','week','month','year','weekday')
date_time_table = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
date_time_table.head()

<ipython-input-61-e2348592ec96>:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  time_data = (dt_table.date_time_pk,dt_table.date_time,t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year,t.dt.weekday)


,date_time_pk,date_time,hour,day,week,month,year,weekday
0,1,2018-04-13 19:00:00,19,13,15,4,2018,4
1,2,2015-08-21 22:00:00,22,21,34,8,2015,4
2,3,2016-08-11 17:00:00,17,11,32,8,2016,3
3,4,2013-09-13 17:00:00,17,13,37,9,2013,4
4,5,2014-07-02 07:00:00,7,2,27,7,2014,2


In [63]:
data.sample()

,date_time,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,address_pk
10,2010-06-17 20:00:00,1,93.092,99.14,47,0.0,0.0,1,12,6,11


In [64]:
data = data.merge(date_time_table, left_on='date_time', right_on='date_time', suffixes=(True,True))
data.drop(['date_time','hour','day','week','month','year','weekday'], axis=1,inplace=True)

In [65]:
data.head()

,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,address_pk,date_time_pk
0,1,82.292,84.650,59,0.56,0.0,1,1,1,1,1
1,1,90.914,97.358,52,0.00,0.0,1,32,1,2,2
2,1,93.434,106.034,58,0.10,0.0,1,1,2,3,3
3,1,91.544,94.712,44,0.00,0.0,1,1,3,4,4
4,1,79.124,79.124,85,0.00,0.0,2,1,4,5,5


In [ ]:
data.head()

# CREATE crime_fact table

In [66]:
data.head()

,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,address_pk,date_time_pk
0,1,82.292,84.650,59,0.56,0.0,1,1,1,1,1
1,1,90.914,97.358,52,0.00,0.0,1,32,1,2,2
2,1,93.434,106.034,58,0.10,0.0,1,1,2,3,3
3,1,91.544,94.712,44,0.00,0.0,1,1,3,4,4
4,1,79.124,79.124,85,0.00,0.0,2,1,4,5,5


In [67]:
crime_fact = data.reset_index()
crime_fact.rename(columns={'index': 'pk'}, inplace=True)
crime_fact.pk = crime_fact.pk +1
crime_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pk              100 non-null    int64  
 1   offenses        100 non-null    int64  
 2   temp            100 non-null    float64
 3   feels_like      100 non-null    float64
 4   humidity_per    100 non-null    int64  
 5   rain_vol_1h_mm  100 non-null    float64
 6   snow_vol_1h_mm  100 non-null    float64
 7   offense_pk      100 non-null    int64  
 8   beat_pk         100 non-null    int64  
 9   premise_pk      100 non-null    int64  
 10  address_pk      100 non-null    int64  
 11  date_time_pk    100 non-null    int64  
dtypes: float64(4), int64(8)
memory usage: 9.5 KB


In [68]:
crime_fact.head()

,pk,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,address_pk,date_time_pk
0,1,1,82.292,84.650,59,0.56,0.0,1,1,1,1,1
1,2,1,90.914,97.358,52,0.00,0.0,1,32,1,2,2
2,3,1,93.434,106.034,58,0.10,0.0,1,1,2,3,3
3,4,1,91.544,94.712,44,0.00,0.0,1,1,3,4,4
4,5,1,79.124,79.124,85,0.00,0.0,2,1,4,5,5


# UPLOAD

In [79]:
def upload_table(table_df,upload_table_sql):
    conn = psycopg2.connect(database=db_name,user=db_user,
    password=db_pass,host=db_host,port=db_port)
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    for i in table_df.values.tolist():
        cur.execute(upload_table_sql, i)
        conn.commit()
    conn.close()


In [97]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [102]:
df

NameError: name 'df' is not defined

In [103]:
namestr(premise_table, globals())


['premise_table', '_70']

In [ ]:
num_files = len(premise_table.values.tolist())

# iterate over files and process
for i, datafile in enumerate(all_files, 1):
    func(cur, datafile)
    conn.commit()
    print('{}/{} files processed.'.format(i, num_files))

In [90]:
count_files = len(premise_table.values.tolist())
files = premise_table.values.tolist()
for count, value in enumerate(files):
    #print(count, value)
    print('{}/{} files processed.'.format(count, count_files))

0/40 files processed.
1/40 files processed.
2/40 files processed.
3/40 files processed.
4/40 files processed.
5/40 files processed.
6/40 files processed.
7/40 files processed.
8/40 files processed.
9/40 files processed.
10/40 files processed.
11/40 files processed.
12/40 files processed.
13/40 files processed.
14/40 files processed.
15/40 files processed.
16/40 files processed.
17/40 files processed.
18/40 files processed.
19/40 files processed.
20/40 files processed.
21/40 files processed.
22/40 files processed.
23/40 files processed.
24/40 files processed.
25/40 files processed.
26/40 files processed.
27/40 files processed.
28/40 files processed.
29/40 files processed.
30/40 files processed.
31/40 files processed.
32/40 files processed.
33/40 files processed.
34/40 files processed.
35/40 files processed.
36/40 files processed.
37/40 files processed.
38/40 files processed.
39/40 files processed.


# upload premise

In [76]:
total_rows = len(premise_table.values.tolist())
for i in premise_table.values.tolist():
    print(f'uploading:{}/{total}')
    cur.execute(premise_table_insert, i)
    conn.commit()

In [81]:
for i in premise_table.values.tolist():
    print(i)

[1, 'restaurant or cafeteria parking lot']
[2, 'other  unknown  or not listed']
[3, 'bar night club parking lot']
[4, 'unk']
[5, 'other unknown']
[6, 'driveway']
[7, 'convenience store']
[8, 'other parking lot']
[9, 'apartment parking lot']
[10, 'grocery supermarket parking lot']
[11, 'department discount store']
[12, 'residence house']
[13, 'strip business center parking lot']
[14, 'grocery supermarket']
[15, 'apartment']
[16, 'hotel motel parking lot']
[17, 'bar night club']
[18, 'service or gas station']
[19, 'vacant single occupancy residence houses townhouses duplexes  etc.']
[20, 'high school']
[21, 'hotel or motel parking lot']
[22, 'libraries  museums']
[23, 'hospital']
[24, 'commercial parking lot or garage']
[25, 'department or discount store']
[26, 'road street sidewalk']
[27, 'residence or house']
[28, 'restaurant cafeteria parking lot']
[29, 'gym  recreat  club house  indoor pool  spa']
[30, 'vacant building commercial']
[31, 'service gas station']
[32, 'pawn  resale shop 

In [80]:
upload_table(premise_table, premise_table_insert)